In [2]:
import numpy as np
import pandas as pd

In [3]:
data = pd.read_csv("breast-cancer-wisconsin.data")

In [4]:
data.head()

,Sample code number,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,Class
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2


In [5]:
data = data.drop("Sample code number",axis=1)
data['Class'] = data['Class'].map({2:0,4:1})

In [6]:
X = data.drop("Class",axis=1)
t = data['Class']

In [7]:
X['constant']  =1

In [9]:
X.replace('?',-99999,inplace=True)

In [10]:
X['Bare Nuclei'] = X['Bare Nuclei'].astype(np.int)

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
X_train, X_test, t_train, t_test = train_test_split(X, t, test_size = 0.2, random_state = 1)

In [13]:
def sigmoid(a):
    return (1/(1+np.exp(-a)))

In [14]:
def weigh_matrix(X,w):
    R = []
    X = np.array(X)
    for x in X:
        y = sigmoid(np.dot(x,w))
        R.append(np.dot(y,1-y))
    return np.diag(R)

In [15]:
#iterative rewieghted least squares
def IRLS(X,t,w):
    R = weigh_matrix(X,w)
    y = sigmoid(np.dot(X,w))
    matrix1 = np.linalg.inv(np.dot(X.T,np.dot(R,X)))
    matrix2 = np.dot(X.T,(y-t))
    w_new = w - np.dot(matrix1,matrix2)

    return w_new

In [16]:
w =np.zeros((10,),dtype=int)
for i in range(50):
    w = IRLS(X_train,t_train,w)
print(w)

[ 4.93924812e-01  7.76270251e-02  5.69516173e-01  2.79381235e-01
  1.51735508e-01  6.28629357e-05  5.89757034e-01  1.93784752e-01
  4.72826628e-01 -9.81795850e+00]


In [17]:
pred = sigmoid(np.dot(X_test,w))

In [18]:
pred[pred<0.5]=0
pred[pred>0.5]=1

In [19]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(t_test, pred)
print(cm)
accuracy_score(t_test, pred)

[[91  1]
 [ 4 44]]


0.9642857142857143